This notebook finetunes the Luke model on the named entity recognition (NER) task: identifying organizations mentioned in news. It consists of two parts:


1.   Clean and preprocess data for the model's use
2.   Finetune the model



**Part 1: data cleaning & preprocessing**

In [108]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os
import pandas as pd
import unicodedata
import numpy as np
import spacy
import gc
from tqdm import tqdm, trange
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from collections import defaultdict

In [2]:
os.chdir('/content/drive/MyDrive/capstone')

In [3]:
data = pd.read_csv('data.csv')

In [4]:
data = data.sort_values(['Sentence', 'Company A', 'Company B'])
all_sentences = data['Sentence'].tolist()
company_a = data['Company A'].tolist()
company_a = [co.strip() for co in company_a]
company_b = data['Company B'].tolist()
company_b = [co.strip() for co in company_b]

In [5]:
'''
In the provided dataset, some examples share the same sentence.
For the NER task, we need to combine those examples and label all organizations that occur in the same sentence.
'''
orgs = []
sent_orgs = set([company_a[0], company_b[0]])
sentences = [all_sentences[0].strip()]
for i, sent in enumerate(all_sentences):
    if i == 0:
        continue
    if sent != all_sentences[i - 1]:
        sentences.append(sent.strip())
        assert(len(sent_orgs) >= 2)
        orgs.append(sent_orgs)
        sent_orgs = set()
    sent_orgs.add(company_a[i])
    sent_orgs.add(company_b[i])
assert(len(sent_orgs) >= 2)
orgs.append(sent_orgs)
assert(len(sentences) == len(orgs))

In [6]:
'''
Define global variables, including model hyperparameters that are relevant
during the dataset creation phase.
'''
label_list = ['O', 'ORG']
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

SEED = 0
# each example contains at most MAX_ENTITY_LENGTH entities
MAX_ENTITY_LENGTH = 1024
# the model only considers spans that are at most ENTITY_WORD_LIMIT words
ENTITY_WORD_LIMIT = 10
# each example contains at most MAX_LENGTH tokens 
# here, "token" is at sub-word level, created from spacy tokenization & LukeTokenizer
MAX_LENGTH = 512

In [7]:
nlp = spacy.load("en_core_web_sm")
docs = list(nlp.pipe(sentences))

In [8]:
'''
Create a list of dict (one dict per doc) from the docs (created by spacy) and their corresponding organizations.
Each dict consists of the following:
text, 
words in the text, 
all spans to be considered by the model indexed at character level ("entity_spans") and word level ("original_word_spans"),
labels that correspond to each span (as indices in label_list).
'''
examples = []
skipped = 0
total_org_count = 0
org_word_len = []
for doc_i, doc in tqdm(enumerate(docs)):
    text = doc.text
    entity_spans = []
    original_word_spans = []
    words = [token.text for token in doc]
    labels = []
    org_occur = set()
    for token_start in doc:
        if len(entity_spans) == MAX_ENTITY_LENGTH:
            break
        for token_end in doc[token_start.i:token_start.i + ENTITY_WORD_LIMIT]:
            entity_start, entity_end = token_start.idx, token_end.idx + len(token_end)
            entity_spans.append((entity_start, entity_end))
            original_word_spans.append((token_start.i, token_end.i + 1))
            span = text[entity_start:entity_end]
            if span in orgs[doc_i]:
                labels.append(1)
                org_word_len.append(token_end.i - token_start.i + 1)
                org_occur.add(span)
            else:
                labels.append(0)
            # once MAX_ENTITY_LENGTH spans are accumulated for an example, we stop adding additional examples
            # data exploratory analysis shows that this truncation leads us to lose 2 valid examples
            if len(entity_spans) == MAX_ENTITY_LENGTH:
                break
    # if an example does not have at least 2 unique organization appearances, we exclude it
    if len(org_occur) < 2:
        skipped += 1
        continue
    total_org_count += sum(labels)
    examples.append(dict(text=text, words=words, entity_spans=entity_spans, original_word_spans=original_word_spans, labels=labels))
print(f"\n{skipped} sentences were skipped since less than 2 organizations were found in these sentences.\n" 
      + f"{len(examples)} sentences containing {total_org_count} organization occurrences remain.")
print(f"The longest organization contains {max(org_word_len)} words.")

253it [00:00, 915.13it/s]


26 sentences were skipped since less than 2 organizations were found in these sentences.
227 sentences containing 658 organization occurrences remain.
The longest organization contains 8 words.


In [9]:
train_examples, val_examples = train_test_split(examples, test_size=0.2, random_state=SEED)

**Part 2: model finetuning**

In [10]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import torch
from transformers import LukeTokenizer, LukeForEntitySpanClassification, Trainer, TrainingArguments, set_seed
from datasets import Dataset, load_metric

In [12]:
set_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [13]:
# may want to experiment with "studio-ousia/luke-large-finetuned-conll-2003"
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-base", max_entity_length=MAX_ENTITY_LENGTH, task="entity_span_classification")

In [14]:
train_dataset = Dataset.from_list(train_examples)
val_dataset = Dataset.from_list(val_examples)

In [15]:
'''
Use LukeTokenizer to tokenize the dataset
'''
def tokenize(example):
    tokenized_inputs = tokenizer(
        text=example['text'], 
        entity_spans=[tuple(span) for span in example['entity_spans']], 
        max_length=MAX_LENGTH,
        max_entity_length=MAX_ENTITY_LENGTH,
        truncation=True,
        return_tensors='pt'
    )

    for k, v in tokenized_inputs.items():
        if isinstance(v, torch.Tensor) and v.shape[0] == 1:
            tokenized_inputs[k] = v.squeeze()

    tokenized_inputs["labels"] = torch.tensor(example['labels'])
    return tokenized_inputs

In [16]:
tokenized_train_data = train_dataset.map(tokenize)
tokenized_val_data = val_dataset.map(tokenize)

  0%|          | 0/181 [00:00<?, ?ex/s]

  0%|          | 0/46 [00:00<?, ?ex/s]

In [17]:
'''
Compute precision, recall, f1, and accuracy (for evaluation on the validation dataset during training)
'''
def compute_metrics(p):
    precision = load_metric("precision")
    recall = load_metric("recall")
    f1 = load_metric("f1")
    accuracy = load_metric("accuracy")

    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions, true_labels = [], []
    for prediction, label in zip(predictions, labels):
        for p, l in zip(prediction, label):
            # -100 represents special tokens [CLS] and [SEP] that should be ignored in metric computation
            if l != -100:
                true_predictions.append(p)
                true_labels.append(l)

    precision_score = precision.compute(predictions=true_predictions, references=true_labels, average="macro")["precision"]
    recall_score = recall.compute(predictions=true_predictions, references=true_labels, average="macro")["recall"]
    f1_score = f1.compute(predictions=true_predictions, references=true_labels, average="macro")["f1"]
    accuracy_score = accuracy.compute(predictions=true_predictions, references=true_labels)["accuracy"]
    return {"precision": precision_score, "recall": recall_score, "f1": f1_score, "accuracy": accuracy_score}

In [18]:
# may want to experiment with "studio-ousia/luke-large-finetuned-conll-2003"
model = LukeForEntitySpanClassification.from_pretrained("studio-ousia/luke-base", ignore_mismatched_sizes=True, id2label=id2label, label2id=label2id).to(device)

Some weights of the model checkpoint at studio-ousia/luke-base were not used when initializing LukeForEntitySpanClassification: ['entity_predictions.transform.dense.weight', 'lm_head.layer_norm.bias', 'entity_predictions.transform.LayerNorm.weight', 'entity_predictions.transform.LayerNorm.bias', 'lm_head.dense.weight', 'entity_predictions.bias', 'lm_head.bias', 'entity_predictions.transform.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LukeForEntitySpanClassificati

In [24]:
EPOCHS = 20
LR = 1e-5
WD = 0.01
BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 64

training_args = TrainingArguments(
    # change folder name here, to avoid replacing the previous model's outputs
    output_dir="output/xxx/", 
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=WD,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    load_best_model_at_end=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [26]:
# to prevent CUDA out of memory issues -- if they still exist, restart runtime
torch.cuda.empty_cache()
gc.collect() # run until the number < 100

0

In [ ]:
# define checkpoint to resume training from, if needed
CKPT = None
train_result = trainer.train(resume_from_checkpoint=CKPT)
trainer.save_model()
trainer.save_state()

The following columns in the training set don't have a corresponding argument in `LukeForEntitySpanClassification.forward` and have been ignored: original_word_spans, entity_spans, words, text. If original_word_spans, entity_spans, words, text are not expected by `LukeForEntitySpanClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 181
  Num Epochs = 20
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 64
  Total optimization steps = 40
  Number of trainable parameters = 274506754


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,0.040800,0.023840,0.496585,0.500000,0.498287,0.993171
1,0.036400,0.023404,0.496585,0.500000,0.498287,0.993171
2,0.033200,0.021084,0.913723,0.569349,0.617524,0.993929
3,0.029900,0.020527,0.872362,0.614321,0.674079,0.994214
4,0.028200,0.021289,0.921487,0.597103,0.656751,0.994261
5,0.027400,0.019579,0.870898,0.673205,0.735644,0.994736
6,0.025800,0.018817,0.855943,0.683526,0.741784,0.994688
7,0.023900,0.018300,0.909974,0.662956,0.732556,0.994925
8,0.022700,0.018232,0.881893,0.683646,0.747551,0.994925
9,0.021800,0.018205,0.811890,0.755894,0.781083,0.994593


The following columns in the evaluation set don't have a corresponding argument in `LukeForEntitySpanClassification.forward` and have been ignored: original_word_spans, entity_spans, words, text. If original_word_spans, entity_spans, words, text are not expected by `LukeForEntitySpanClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 46
  Batch size = 1
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to output/checkpoint-2
Configuration saved in output/checkpoint-2/config.json
Model weights saved in output/checkpoint-2/pytorch_model.bin
tokenizer config file saved in output/checkpoint-2/tokenizer_config.json
Special tokens file saved in output/checkpoint-2/spe

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,0.040800,0.023840,0.496585,0.500000,0.498287,0.993171
1,0.036400,0.023404,0.496585,0.500000,0.498287,0.993171
2,0.033200,0.021084,0.913723,0.569349,0.617524,0.993929
3,0.029900,0.020527,0.872362,0.614321,0.674079,0.994214
4,0.028200,0.021289,0.921487,0.597103,0.656751,0.994261
5,0.027400,0.019579,0.870898,0.673205,0.735644,0.994736
6,0.025800,0.018817,0.855943,0.683526,0.741784,0.994688
7,0.023900,0.018300,0.909974,0.662956,0.732556,0.994925
8,0.022700,0.018232,0.881893,0.683646,0.747551,0.994925
9,0.021800,0.018205,0.811890,0.755894,0.781083,0.994593


The following columns in the evaluation set don't have a corresponding argument in `LukeForEntitySpanClassification.forward` and have been ignored: original_word_spans, entity_spans, words, text. If original_word_spans, entity_spans, words, text are not expected by `LukeForEntitySpanClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 46
  Batch size = 1
Saving model checkpoint to output/checkpoint-26
Configuration saved in output/checkpoint-26/config.json
Model weights saved in output/checkpoint-26/pytorch_model.bin
tokenizer config file saved in output/checkpoint-26/tokenizer_config.json
Special tokens file saved in output/checkpoint-26/special_tokens_map.json
added tokens file saved in output/checkpoint-26/added_tokens.json
The following columns in the evaluation set don't have a corresponding argument in `LukeForEntitySpanClassification.forward` and have been ignored: original_word_spans, entity_spans, words, text. If origina

In [107]:
'''
Predict on the validation dataset and save output in CONLL format
'''
# change prediction output path here (put it in the same foler as the model)
output_path = "output/xxx/xxx-val-pred.conll" 
text_list = [e["text"] for e in val_dataset]
words_list = [e["words"] for e in val_dataset]
original_word_spans_list = [e["original_word_spans"] for e in val_dataset]
entity_spans_list = []
for e in val_dataset:
    entity_spans_list.append([tuple(span) for span in e["entity_spans"]])

with open(output_path, "w", encoding="utf-8") as f:
    for i in range(len(text_list)):
        text = text_list[i]
        words = words_list[i]
        entity_spans = entity_spans_list[i]
        original_word_spans = original_word_spans_list[i]

        inputs = tokenizer(
            text=text, 
            entity_spans=entity_spans, 
            max_length=MAX_LENGTH,
            max_entity_length=MAX_ENTITY_LENGTH,
            truncation=True,
            return_tensors='pt'
        )
        inputs = inputs.to(device)
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        max_logits, max_indices = logits[0].max(dim=1)

        predictions = []
        for logit, index, span in zip(max_logits, max_indices, original_word_spans):
            if index != 0:  # the span is not NIL
                predictions.append((logit, span, model.config.id2label[int(index)]))

        # construct an IOB2 label sequence
        predicted_sequence = ["O"] * len(words)
        for _, span, label in sorted(predictions, key=lambda o: o[0], reverse=True):
            if all([o == "O" for o in predicted_sequence[span[0] : span[1]]]):
                predicted_sequence[span[0]] = "B-" + label
                if span[1] - span[0] > 1:
                    predicted_sequence[span[0] + 1 : span[1]] = ["I-" + label] * (span[1] - span[0] - 1)

        for token, label in zip(words, predicted_sequence):
            f.write(f"{token} {label}\n")
        f.write("\n")